# Crear y ejecutar un trabajo de entrenamiento (SDK de Python de Amazon SageMaker)

El SDK de Python de Amazon SageMaker incluye el estimador sagemaker.estimator.Estimator. Puede utilizar esta clase, en el módulo sagemaker.estimator, con cualquier algoritmo.

In [2]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role

In [10]:
role = get_execution_role() #get_execution_role recupera el rol de IAM que creó durante la creación de la instancia de notebooks
region = boto3.Session().region_name
bucket= 'lbk-analytics-dev' # Replace with your s3 bucket name
prefix = 'sagemaker/xgboost-mnist' # Used as part of the path in the bucket where you store data

In [11]:
container = get_image_uri(boto3.Session().region_name, 'xgboost' , '1.0-1')

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


In [12]:
train_data = 's3://{}/{}/{}' .format(bucket, prefix, 'train' )
validation_data = 's3://{}/{}/{}' .format(bucket, prefix, 'validation' )
s3_output_location = 's3://{}/{}/{}' .format(bucket, prefix,'xgboost_model_sdk' )
print(train_data)

s3://lbk-analytics-dev/sagemaker/xgboost-mnist/train


In [13]:
xgb_model = sagemaker.estimator.Estimator(container, 
                                            role,#rol para ejecutar la tarea
                                            train_instance_count= 1 , # numero de instancias para entrenamiento 
                                            train_instance_type= 'ml.m4.xlarge' , #tipo de la instancia
                                            train_volume_size = 5 , #tamaño en GB de EBS
                                            output_path=s3_output_location, # ruta de salida de resultados
                                            sagemaker_session=sagemaker.Session())

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [14]:
#hiperparametros
xgb_model.set_hyperparameters(max_depth = 5 ,
                                eta = .2 ,
                                gamma = 4 ,
                                min_child_weight = 6 ,
                                silent = 0 ,
                                objective = "multi:softmax" ,
                                num_class = 10 ,
                                num_round = 10 )

In [15]:
train_channel = sagemaker.session.s3_input(train_data,content_type= 'text/csv' )
valid_channel = sagemaker.session.s3_input(validation_data,content_type= 'text/csv' )

data_channels = { 'train' : train_channel, 'validation' : valid_channel}

xgb_model.fit(inputs=data_channels, logs= True )



's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-07-28 23:55:20 Starting - Starting the training job...
2020-07-28 23:55:23 Starting - Launching requested ML instances......
2020-07-28 23:56:41 Starting - Preparing the instances for training......
2020-07-28 23:57:43 Downloading - Downloading input data......
2020-07-28 23:58:52 Training - Downloading the training image...
2020-07-28 23:59:11 Training - Training image download completed. Training in progress.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Det

# Implementar el modelo para servicios de alojamiento de Amazon SageMaker (SDK de Python de Amazon SageMaker)

El método deploy crea el modelo que puede implementarse, configura el punto de enlace de los servicios de alojamiento de Amazon SageMaker y lanza el punto de enlace para alojar el modelo.
También devuelve un objeto sagemaker.predictor.RealTimePredictor, que puede utilizar para obtener inferencias desde el modelo

In [ ]:
xgb_predictor = xgb_model.deploy(initial_instance_count= 1 ,instance_type= 'ml.m4.xlarge',)

# Implementar un modelo con la transformación por lotes (biblioteca de Python de alto nivel de Amazon SageMaker)
El siguiente código crea un objeto sagemaker.transformer.Transformer del modelo que capacitó. A continuación, llama al método transform de ese objeto para crear un trabajo de transformación. Cuando se crea el objeto sagemaker.transformer.Transformer, especifique el número y el tipo de las instancias de aprendizaje automático que se va a utilizar para realizar el trabajo de transformación por lotes y la ubicación en Amazon S3 en la que desea almacenar las inferencias

In [ ]:
batch_input = 's3://{}/{}/test/examples' .format(bucket, prefix) # The location of the test dataset
batch_output = 's3://{}/{}/batch-inference' .format(bucket, prefix) # The location to store the results of the batch transform job

transformer = xgb_model.transformer(instance_count= 1, instance_type= 'ml.m4.xlarge' , output_path=batch_output)
transformer.transform(data=batch_input, data_type= 'S3Prefix', content_type= 'text/csv' , split_type= 'Line' )

transformer.wait()

# Validar un modelo implementado en los servicios de alojamiento de Amazon SageMaker (Amazon SageMaker SDK para Phyton)
Para validar el modelo utilizando el Amazon SageMaker SDK para Python, utilice el objeto sagemaker.predictor.RealTimePredictor

In [ ]:
s3 = boto3.resource( 's3' )
test_key = "{}/test/examples" .format(prefix)
s3.Bucket(bucket).download_file(test_key, 'test_data' )

In [ ]:
%matplotlib inline
for i in range ( 0 , 10 ):
    img = test_set[ 0 ][i]
    label = test_set[ 1 ][i]
    img_reshape = img.reshape(( 28 , 28 ))
    imgplot = plt.imshow(img_reshape, cmap= 'gray' )
    print( 'This is a {}' .format(label))
plt.show()

In [ ]:
with open( 'test_data' , 'r' ) as f:
    for j in range( 0 , 10 ):
        single_test = f.readline()
        result = xgb_predictor.predict(single_test)
        print(result)